In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install pdfplumber --no-cache-dir


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.0/12.9 MB 393.8 kB/s eta 0:00:33
     --------------------------------------- 0.1/12.9 MB 573.4 kB/s eta 0:00:23
     --------------------------------------- 0.2/12.9 MB 919.0 kB/s eta 0:00:14
     - -------------------------------------- 0.3/12.9 MB 1.6 MB/s eta 0:00:09
     - -------------------------------------- 0.5/12.9 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/12.9 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.7/12.9 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/12.9 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.8/12.9 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.9/12.9 MB 1.8 MB/s eta 0:00:07
     -- ------------------------------------- 0.9/12.9 


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

In [3]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [4]:
pdf_path = "texts/certificado.pdf"

In [5]:
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"

In [6]:
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [7]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    doc = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return tokens

In [8]:
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['verifique', 'contenido', 'confiabilidad', 'certificado', 'ingresando', 'digite', 'visualice', 'imagen', 'generada', 'momento', 'ilimitada', 'calendario', 'contados', 'fecha', 'atencion', 'comerciante', 'cumplido', 'deber', 'legal', 'renovar', 'matricula', 'mercantil', 'd', 's', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'confianza', 'ean', 'n', 'renueve', 'matricula', 'mercantil', 'tardar', 'marzo', 'v', 'i', 'c', 'n', 'n', 's', 'fundamento', 'inscrripciones', 'efectu', 'adas', 'registro', 'mercantil', 'comercio', 'certiefica']


In [9]:
with open("texts/certificado_limpio.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))